# Strategic Patient Risk Stratification & Readmission Predictive Modeling
## Vitality Health Network (VHN)

**Course:** ITS 2122: Python for Data Science & AI (Semester 3 – 2025)  
**Dataset:** Diabetes 130-US Hospitals (1999–2008)  
**Objective:** Analyze historical hospital data to identify drivers of 30-day readmissions and build a risk stratification system

---

## Table of Contents
1. [Phase 1: Data Ingestion & Clinical Sanitation](#phase1)
2. [Phase 2: Data Enrichment via Web Scraping](#phase2)
3. [Phase 3: Exploratory Data Analysis](#phase3)
4. [Phase 4: Feature Engineering - Vitality Complexity Index](#phase4)

---

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from utils import (
    calculate_vci_score,
    categorize_vci_risk,
    scrape_icd9_description,
    audit_data_quality,
    print_audit_summary,
    plot_readmission_by_category,
    plot_readmission_rate_by_category,
    create_correlation_heatmap
)

# Configure display settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')

print("✓ Libraries imported successfully")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

<a id='phase1'></a>
# Phase 1: Data Ingestion & Clinical Sanitation

In this phase, we perform professional healthcare data cleaning:
- Load and audit the dataset
- Handle missing values and data quality issues
- Remove deceased patients
- Convert data types appropriately
- Remove duplicates
- Document all cleaning decisions with clinical rationale

## 1.1 Load Datasets

In [1]:
# Load main dataset
df = pd.read_csv('data_files/diabetic_data.csv')
print(f"Dataset loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")

# Load ID mappings
id_mapping = pd.read_csv('data_files/IDs_mapping.csv')
print(f"\nID mapping loaded: {id_mapping.shape[0]} mappings")
print("\nFirst few rows of the dataset:")
df.head()

NameError: name 'pd' is not defined

## 1.2 Initial Data Audit

In [ ]:
# Display dataset information
print("=" * 70)
print("DATASET STRUCTURE")
print("=" * 70)
df.info()

In [ ]:
# Statistical summary
print("\n" + "=" * 70)
print("STATISTICAL SUMMARY - NUMERICAL FEATURES")
print("=" * 70)
df.describe()

In [ ]:
# Check for '?' values (common placeholder in healthcare data)
print("\n" + "=" * 70)
print("CHECKING FOR '?' PLACEHOLDER VALUES")
print("=" * 70)

question_mark_counts = {}
for col in df.columns:
    if df[col].dtype == 'object':
        count = (df[col] == '?').sum()
        if count > 0:
            question_mark_counts[col] = count

if question_mark_counts:
    qm_df = pd.DataFrame.from_dict(question_mark_counts, orient='index', columns=['Count'])
    qm_df['Percentage'] = (qm_df['Count'] / len(df) * 100).round(2)
    qm_df = qm_df.sort_values('Count', ascending=False)
    print(qm_df)
else:
    print("No '?' values found")

## 1.3 Convert '?' to NaN

**Clinical Rationale:** The '?' symbol is used as a placeholder for missing data in many healthcare datasets. Converting these to NaN allows proper handling with pandas missing data methods and prevents these values from being treated as valid categories.

In [ ]:
# Replace '?' with NaN
df.replace('?', np.nan, inplace=True)
print("✓ Converted all '?' values to NaN")

# Verify conversion
print(f"\nTotal NaN values in dataset: {df.isna().sum().sum():,}")

## 1.4 Comprehensive Data Quality Audit

In [ ]:
# Perform audit using utility function
audit_results = audit_data_quality(df)
print_audit_summary(audit_results)

## 1.5 Handle High Missingness Columns

**Clinical Rationale:** Columns with >90% missing data provide minimal analytical value and can introduce bias. Common examples in healthcare data:
- **weight**: Often not recorded consistently across facilities
- **payer_code**: May not be captured in all systems
- **medical_specialty**: Frequently missing in administrative data

We document these as **data quality limitations** rather than attempting imputation, which would be clinically inappropriate.

In [ ]:
# Identify columns with >90% missingness
high_missing_threshold = 0.90
missing_pct = df.isna().sum() / len(df)
high_missing_cols = missing_pct[missing_pct > high_missing_threshold].index.tolist()

print(f"Columns with >{high_missing_threshold*100}% missing data:")
for col in high_missing_cols:
    pct = missing_pct[col] * 100
    print(f"  • {col}: {pct:.2f}% missing")

# Drop these columns
if high_missing_cols:
    df.drop(columns=high_missing_cols, inplace=True)
    print(f"\n✓ Dropped {len(high_missing_cols)} columns with extreme missingness")
    print(f"New dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
else:
    print("\nNo columns exceed the 90% missingness threshold")

## 1.6 Remove Deceased Patients

**Clinical Rationale:** Patients who expired (died) during hospitalization cannot be readmitted and should be excluded from readmission analysis. Including them would:
1. Artificially inflate the "no readmission" group
2. Skew risk models toward end-of-life care patterns
3. Violate the clinical definition of readmission risk

We use the discharge_disposition_id mapping to identify deceased patients.

In [ ]:
# Display discharge disposition mappings
print("Discharge Disposition ID Mappings:")
print(id_mapping[id_mapping['Table'] == 'discharge_disposition_id'])

# Identify deceased patient codes
# Typically codes 11, 13, 14, 19, 20, 21 indicate expired/hospice/deceased
deceased_codes = [11, 13, 14, 19, 20, 21]

print(f"\nDeceased/Expired discharge codes: {deceased_codes}")
print(f"Patients before removal: {len(df):,}")

# Count deceased patients
deceased_count = df[df['discharge_disposition_id'].isin(deceased_codes)].shape[0]
print(f"Deceased patients identified: {deceased_count:,} ({deceased_count/len(df)*100:.2f}%)")

# Remove deceased patients
df = df[~df['discharge_disposition_id'].isin(deceased_codes)].copy()
print(f"Patients after removal: {len(df):,}")
print(f"\n✓ Removed {deceased_count:,} deceased patients from analysis")

## 1.7 Convert to Appropriate Data Types

**Clinical Rationale:** Proper data typing improves:
- Memory efficiency
- Analysis accuracy
- Categorical analysis capabilities

In [ ]:
# Define categorical columns
categorical_cols = [
    'race', 'gender', 'age', 'admission_type_id', 'discharge_disposition_id',
    'admission_source_id', 'max_glu_serum', 'A1Cresult',
    'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
    'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',
    'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
    'insulin', 'glyburide-metformin', 'glipizide-metformin',
    'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone',
    'change', 'diabetesMed', 'readmitted'
]

# Convert to category dtype
for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].astype('category')

print("✓ Converted categorical columns to 'category' dtype")
print(f"\nMemory usage after conversion: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 1.8 Remove Duplicate Records

**Clinical Rationale:** Duplicate records can occur due to:
- Data entry errors
- System integration issues
- Multiple submissions

Duplicates must be removed to ensure accurate statistical analysis and prevent bias in predictive models.

In [ ]:
# Check for duplicates
duplicate_count = df.duplicated().sum()
print(f"Duplicate rows found: {duplicate_count:,}")

if duplicate_count > 0:
    df.drop_duplicates(inplace=True)
    print(f"✓ Removed {duplicate_count:,} duplicate rows")
else:
    print("✓ No duplicate rows found")

print(f"\nFinal dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

## 1.9 Phase 1 Summary

**Data Cleaning Decisions Summary:**

In [ ]:
print("=" * 70)
print("PHASE 1: DATA SANITATION COMPLETE")
print("=" * 70)
print("\nCleaning Actions Performed:")
print("  1. ✓ Converted '?' placeholders to NaN")
print(f"  2. ✓ Dropped {len(high_missing_cols) if high_missing_cols else 0} columns with >90% missingness")
print(f"  3. ✓ Removed {deceased_count:,} deceased patients")
print("  4. ✓ Converted categorical columns to appropriate dtype")
print(f"  5. ✓ Removed {duplicate_count:,} duplicate records")
print(f"\nFinal Clean Dataset: {df.shape[0]:,} rows × {df.shape[1]} columns")
print("=" * 70)